*Utilização do modelo Vision Transformers para Classificação de Estágio de Glaucoma

*Otimização em busca dos melhores Hiperparâmetros para o classificador

*CNNs


# Dependências

In [ ]:
#pip install -U tensorflow-addons

In [ ]:
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install classification-models-3D --quiet
#pip install classification-models-3D
!pip install keras_applications --quiet
!pip install optuna --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 16.7 MB/s eta 0:00:00


In [23]:
import pandas as pd
import numpy as np
import tensorflow as tf
#import tensorflow_addons as tfa
import glob, warnings
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from collections import Counter#usar para contar a quantidade de uma classe ex: counter(y_train)
import os
from google.colab import drive
from google.colab import files
import keras
import zipfile
from io import BytesIO

from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D, Concatenate, Reshape, GlobalMaxPooling2D, GlobalMaxPooling3D, GlobalAveragePooling3D, Conv2D, Conv1D, Add
from keras import regularizers
from tensorflow.keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
import sklearn
from sklearn.metrics import cohen_kappa_score, roc_auc_score, roc_curve,accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import random
import cv2
import gdown
import matplotlib.pyplot as plt
import requests
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import random
import sklearn
from sklearn.model_selection import StratifiedKFold, KFold
from tensorflow.keras import layers
import csv
from scipy import stats
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import DenseNet169
from keras.applications.resnet import ResNet50
from keras.applications.resnet import ResNet101
from keras.applications.resnet import ResNet152
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg19 import VGG19
from keras.models import Sequential, Model
import time
from pathlib import Path
import gc
#from keras.backend import int_shape
from tensorflow.keras import backend as K
import pickle
#from livelossplot import PlotLossesKeras
import albumentations as A
import optuna
from keras.applications.densenet import preprocess_input
import json
from keras import backend as K
from classification_models_3D.tfkeras import Classifiers
#keras.__version__
warnings.filterwarnings('ignore')
from matplotlib import pyplot
import sys
#print('TensorFlow Version ' + tf.__version__)

In [4]:
#obtendo diretorio da base
drive.mount('/content/gdrive',force_remount=True)
path_dataset = '/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/multi-modality_images'
os.chdir(path_dataset)

#Funções Especificas GAMMA
from utils_GAMMA import padroniza_resultado
from utils_GAMMA import converte

Mounted at /content/gdrive


# Configuração dos Hiperparâmetros

In [5]:
# DATA
# AUGMENTATION
def define_hp():#definir hiperparâmetros
  IMAGE_SIZE = 128
  depth = 64 #Profundidade dos volumes OCTs (Usado na leitura de volumes OCTs)

  EPOCHS = 3
  early_stop_epochs = 15
  learning_rate_epochs = 5
  number_of_random_points = 25  # random searches to start opt process
  return IMAGE_SIZE, depth, EPOCHS, early_stop_epochs, learning_rate_epochs, number_of_random_points

def create_folder_save(dataset):#Criar pastas para salvar modelos treinados e criados pelo processo de otimização
  path_to_models = '/content/Modelos Salvos/'
  if not Path(path_to_models).is_dir():
    os.mkdir(path_to_models)

  dir_save = path_to_models + dataset
  if not Path(dir_save).is_dir():
    os.mkdir(dir_save)
  return dir_save

def create_folder_results_optuna(dataset):#Criar pastas para salvar dataframe criados pelo processo de otimização
  path_to_results_optuna = '/content/Resultados_optuna/'
  if not Path(path_to_results_optuna).is_dir():
    os.mkdir(path_to_results_optuna)

  results_dir = path_to_results_optuna + dataset
  if not Path(results_dir).is_dir():
    os.mkdir(results_dir)
  return results_dir

# Leitura de OCTs

In [ ]:
path_dataset_Novas_OCTs = '/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data'

In [ ]:
# from utils_GAMMA import process_scan
# from utils_GAMMA import dir_octs
from utils_GAMMA_Novas_OCTS import process_scan
from utils_GAMMA_Novas_OCTS import dir_octs
#scan_paths = dir_octs(path_dataset,'multi-modality_images',0)#obtem diretorio dos volumes
scan_paths = dir_octs(path_dataset_Novas_OCTs,'Novas Imagens OCT',0)#Imagens com recorte
#scan_paths = scan_paths[:2]
X_oct = np.array([process_scan(path,IMAGE_SIZE, depth) for path in scan_paths])#lê volumes

/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/Novas Imagens OCT/0001
/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/Novas Imagens OCT/0002
/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/Novas Imagens OCT/0003
/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/Novas Imagens OCT/0004
/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/Novas Imagens OCT/0005
/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/Novas Imagens OCT/0006
/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/Novas Imagens OCT/0007
/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/Novas Imagens OCT/0008
/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/Novas Imagens OCT/0009
/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/Novas Imagens OCT/0010
/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/Novas Imagens OCT/0011
/content/gdrive/My Drive/Doutorado/Bases/GAMMA_training_data/Novas Imagens O

In [ ]:
with open('Novas_OCT_64_128_Train.pkl','wb') as f:#Depth - IMG Size
  pickle.dump(X_oct, f)

# Carregar OCTs Salvas

In [6]:
def load_dataset():
  folder_OCTs_labels = '/content/gdrive/MyDrive/Doutorado/Bases/GAMMA_training_data/multi-modality_images/'
  file_octs = 'Novas_OCT_64_128_Train.pkl'
  #folder_labels = '/content/gdrive/MyDrive/Doutorado/Bases/GAMMA_training_data/'
  file_labels = 'Y_Train.pkl'
  #Carregar array
  with open(folder_OCTs_labels + file_octs,'rb') as f:
    X_train_oct = pickle.load(f)

  with open(folder_OCTs_labels + file_labels,'rb') as f:
    Y = pickle.load(f)
  num_classes = len(np.unique(Y))
  return X_train_oct, Y, num_classes

# Divide Treino/Validação
Usar split de validação fixo

In [7]:
def split_train_val(X_train_oct, Y):
  x_train_oct, x_val_oct, y_train_oct, y_val_oct = train_test_split(X_train_oct, Y, test_size=0.1, random_state=42,stratify=Y)

  print(f"x_train shape: {x_train_oct.shape} - y_train shape: {y_train_oct.shape}")
  print(f"x_val shape: {x_val_oct.shape} - y_val shape: {y_val_oct.shape}")
  return x_train_oct, x_val_oct, y_train_oct, y_val_oct

# Fit Models

In [9]:
def fit_models(number_trial, fine_tuning, model, batch_size,lr, dir_save, x_train_oct, EPOCHS, early_stop_epochs, y_train_oct, x_val_oct, y_val_oct, cnn_model):
  # callbacks for early stopping and for learning rate reducer
  fn = dir_save + cnn_model + '_' +number_trial + '_cnn.keras'
  #fn = dir_save + str(trial.number) + '_cnn.hdf5'#somente pesos
  #fn = '/content/Modelos Salvos/' + str(trial.number) + '_cnn.hdf5'#somente pesos


  # If already batched
  #total_steps = len(train_set)*config['EPOCHS']
  # If not batched
  #total_steps = len(train_set)/config['BATCH_SIZE']*config['EPOCHS']
  total_steps = len(x_train_oct)/batch_size*EPOCHS
  # 5% of the steps
  warmup_steps = int(0.05*total_steps)

  #compile
  model.trainable = True
  opt = Adam(learning_rate=lr)
  #opt = Adam(learning_rate=1e-5)  # default = 0.001 // learning_rate=lr
  #model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])#loss='sparse_categorical_crossentropy'
  model.compile(loss="sparse_categorical_crossentropy",
                  optimizer='adam',
                  #jit_compile=True,
                  metrics=['accuracy'])

  if fine_tuning==1:# 2 etapa de treino. todas as camadas da cnn descongeladas
    callbacks_list = [EarlyStopping(monitor='val_loss', patience=early_stop_epochs, verbose=1, mode='auto', restore_best_weights=True),
                      #ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=learning_rate_epochs, verbose=0, mode='auto', min_lr=1.0e-6),
                      ModelCheckpoint(filepath=fn, monitor='val_loss', verbose=1, save_best_only=True),
                      #WarmupCosineDecay(total_steps=total_steps, warmup_steps=warmup_steps, hold=int(warmup_steps/2), start_lr=0.0, target_lr=lr)
                      ]
  else:#aquecimento
    callbacks_list = [EarlyStopping(monitor='val_loss', patience=early_stop_epochs, verbose=1, mode='auto', restore_best_weights=True),
                      #ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=learning_rate_epochs, verbose=0, mode='auto', min_lr=1.0e-6),
                      ModelCheckpoint(filepath=fn, monitor='val_loss', verbose=1,save_weights_only=False, save_best_only=True)
                      ]


  # fit the model
  h = model.fit(x=x_train_oct, y=y_train_oct,
                          batch_size=batch_size,
                          epochs=EPOCHS,
                          #validation_split=0.1,
                          validation_data=(x_val_oct, y_val_oct),
                          shuffle=True, verbose=1,
                          callbacks=callbacks_list)

  if fine_tuning==1:
    lrs = callbacks_list[2].lrs#Conjunto de taxas de aprendizado para ser possível plotar o grafico
    return h, model, lrs
  else:
    return h, model

# Build Model

In [10]:
def build_model(num_dense_nodes, lr, dense_nodes_divisor, batch_size, drop_out_rate, cnn_model, num_layers, depth, IMAGE_SIZE, num_classes):
  canais = 3
  if cnn_model == 'InceptionV3':
    Rede3D, preprocess_input = Classifiers.get('inceptionv3')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, 3), weights='imagenet',include_top=False)

  if cnn_model == 'VGG19':
    Rede3D, preprocess_input = Classifiers.get('vgg19')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, 3), weights='imagenet',include_top=False)

  if cnn_model == 'resnet50':
    #x = tf.keras.applications.resnet50.preprocess_input(x)
    Rede3D, preprocess_input = Classifiers.get('resnet50')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, 3), weights='imagenet',include_top=False)

  if cnn_model == 'resnet101':
    Rede3D, preprocess_input = Classifiers.get('resnet101')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, 3), weights='imagenet',include_top=False)

  if cnn_model == 'resnet152':
    Rede3D, preprocess_input = Classifiers.get('resnet152')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, 3), weights='imagenet',include_top=False)

  if cnn_model == 'Densenet121':
    #x = tf.keras.applications.densenet.preprocess_input(x)
    Rede3D, preprocess_input = Classifiers.get('densenet121')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, 3), weights='imagenet',include_top=False)

  if cnn_model == 'Densenet169':
    #x = tf.keras.applications.densenet.preprocess_input(x)
    Rede3D, preprocess_input = Classifiers.get('densenet121')
    model_oct = Rede3D(input_shape=(depth, IMAGE_SIZE, IMAGE_SIZE, 3), weights='imagenet',include_top=False)

  x = model_oct.output
  x = Flatten()(x)
  #MLP
  #x = Dense(num_dense_nodes, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4), activation='relu')(x)#regularazação
  x = Dense(num_dense_nodes, activation='relu')(x)
  x = Dropout(drop_out_rate)(x)
  if num_layers == 2:
    #x = Dense(num_dense_nodes//dense_nodes_divisor, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4),activation='relu')(x)#regularazação
    x = Dense(num_dense_nodes//dense_nodes_divisor, activation='relu')(x)
    x = Dropout(drop_out_rate)(x)
  output_tensor = Dense(num_classes, activation='softmax')(x)
  #Instanciar e compilar modelo
  model = Model(inputs=model_oct.input,outputs=output_tensor)
  opt = Adam(learning_rate=lr)  # default = 0.001 // learning_rate=lr
  model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])#loss='sparse_categorical_crossentropy'

  return model

# Configuração do Otimizador do classificador OCT

In [11]:
def objective(trial):#cria e avalia modelo
  num_dense_nodes = trial.suggest_categorical('num_dense_nodes', [32, 64, 128, 256])
  dense_nodes_divisor = trial.suggest_categorical('dense_nodes_divisor',[2, 4, 8])
  lr = trial.suggest_categorical('lr', [0.01, 0.001, 0.0001])
  batch_size = trial.suggest_categorical('batch_size', [1, 2, 3])
  drop_out_rate=trial.suggest_float('drop_out_rate', 0.0, 0.5, step=0.1)
  num_layers = trial.suggest_categorical('num_layers',[1,2])
  #drop_out=trial.suggest_discrete_uniform('drop_out', 0.05, 0.5, 0.05)
  cnn_model = trial.suggest_categorical('cnn_model', ['VGG19', 'resnet50', 'Densenet121', 'Densenet169'])

  #Hiperparâmetros
  IMAGE_SIZE, depth, EPOCHS, early_stop_epochs, learning_rate_epochs, number_of_random =  define_hp()
  dataset = 'GAMMA/'
  dir_save = create_folder_save(dataset)#Pastas para salvar Modelos treinados e dataframe oriundo do processo de otimização

  X_train_oct, Y, num_classes = load_dataset()#Load oct volumes
  x_train_oct, x_val_oct, y_train_oct, y_val_oct = split_train_val(X_train_oct, Y)#Split train/val

  #constroi modelo
  model = build_model(num_dense_nodes, lr, dense_nodes_divisor, batch_size, drop_out_rate, cnn_model, num_layers, depth, IMAGE_SIZE, num_classes)

  print("="*10, "Trial", str(trial.number), "="*10, cnn_model)


  #Fine tuning
  fine_tuning=0 # (1) - Com cosine decay
  # for layer in model.layers:
    # print(layer.trainable)
  if fine_tuning==1:
    h,model,lrs = fit_models(str(trial.number), fine_tuning, model, batch_size,lr, dir_save, x_train_oct, EPOCHS, early_stop_epochs, y_train_oct, x_val_oct, y_val_oct, cnn_model)
    plt.plot(lrs)
  else:
    h,model = fit_models(str(trial.number), fine_tuning, model, batch_size,lr, dir_save, x_train_oct, EPOCHS, early_stop_epochs, y_train_oct, x_val_oct, y_val_oct, cnn_model)

  validation_loss = np.min(h.history['val_loss'])
  val_acc = np.max(h.history['val_accuracy'])

  #save history in json file
  history_dict = h.history
  history_dir = '/content/history/'

  if not Path(history_dir).is_dir():
    os.mkdir(history_dir)

  out_file=open(history_dir + 'history_json_Trial_' + (str(trial.number)),"a")
  json.dump(history_dict, out_file)
  out_file.close()

  #json.dump(history_dict, open(history_path, 'w'))
  return validation_loss, val_acc

#Otimização

In [14]:
# if not Path(dir_save).is_dir():
#   os.mkdir(dir_save)

# os.chdir(dir_save)
def otimizacao():
  optimizer_direction = ['minimize', "maximize"]
  print('\n*** starting at',pd.Timestamp.now())
  start_time_total = time.time()
  optuna.logging.set_verbosity(optuna.logging.WARNING)
  study = optuna.create_study(directions=optimizer_direction,study_name="starter-experiment")
  study.optimize(objective, n_trials=1,gc_after_trial=True)
  df_results = study.trials_dataframe()
  elapsed_time_total = (time.time()-start_time_total)/60
  print('\n\ntotal elapsed time =',elapsed_time_total,' minutes')
  return study, df_results

In [15]:
def save_results(study, df_results, results_dir):
  df = study.trials_dataframe()
  df_results.to_csv(results_dir + 'df_optuna_results.csv')
  df_results.to_pickle(results_dir + 'df_optuna_results.pkl')

In [43]:
def sort_results(results_dir):
  dataframe = pd.read_csv(results_dir+'df_optuna_results.csv')
  dataframe.sort_values("values_0",axis=0,ascending=True, inplace=True, na_position='first')
  print("Sorted CSV file (according to multiple columns) = ", dataframe.head(10))
  return dataframe

In [24]:
def summarize_diagnostics(history_no_warm):
 # plot loss train
 fig, ax = pyplot.subplots(2,2, figsize=(10,10))
 fig.tight_layout(pad=2.0)
 ax[0, 0].title.set_text('Cross Entropy Loss Training')
 #ax[0, 0].plot(history_warm['loss'], color='blue', label='Trial_8')
 ax[0, 0].plot(history_no_warm['loss'], color='orange', label='Trial_2')
 ax[0, 0].legend(loc='best')

 # plot loss validation
 ax[0, 1].title.set_text('Cross Entropy Loss Validation')
 #ax[0, 1].plot(history_warm['val_loss'], color='blue', label='Trial_8')
 ax[0, 1].plot(history_no_warm['val_loss'], color='orange', label='Trial_2')
 ax[0, 1].legend(loc='best')

 # plot accuracy training
 #pyplot.subplot(212)
 ax[1, 0].title.set_text('Accuracy Training')
 #ax[1, 0].plot(history_warm['accuracy'], color='blue', label='Trial_8')
 ax[1, 0].plot(history_no_warm['accuracy'], color='orange', label='Trial_2')
 ax[1, 0].legend(loc='best')

  # plot acc validation
 ax[1, 1].title.set_text('Accuracy Validation')
 #ax[1, 1].plot(history_warm['val_accuracy'], color='blue', label='Trial_8')
 ax[1, 1].plot(history_no_warm['val_accuracy'], color='orange', label='Trial_2')
 ax[1, 1].legend(loc='best')

In [51]:
def main():
  study, df_results = otimizacao()
  dataset = 'GAMMA/'
  dir_save = create_folder_save(dataset)#Pastas para salvar Modelos treinados e dataframe oriundo do processo de otimização
  results_dir = create_folder_results_optuna(dataset)
  save_results(study, df_results, results_dir)
  dataframe = sort_results(results_dir)
  history = json.load(open('/content/history/history_json_Trial_0', 'r'))
  summarize_diagnostics(history)

In [ ]:
if __name__ == "__main__":
    main()